### 06 Guided ECCI acquisition on the SEM
This notebook can be used during ECCI acquisiton on SEM to guide the stage tilt to two beam conditions.   

#### Import dependent packages

In [1]:
import sys
sys.path.append("..")

import os
import kikuchipy as kp
from openECCI import io, optimization
from pathlib import Path

#### Load the four optimized correction coefficients calculated from 01_orientation_calibration notebook
__Note__: It is critical to take a new reference ECP pattern from the Si[001] wafer every time after the sample is loaded in SEM. The correction coefficients should be calcultated based on this fresh ECP, so as to minimize the small deviations in sample mounting orientation. 

In [2]:
optimized_correction_coefficients = {'tiltX_corr_angle': -0.5312103261068537,
                                     'tiltY_corr_angle': 1.2285403872457656, 
                                     'tiltZ_corr_angle': 0.06380671840706255, 
                                     'PCz': 3.890243796249578}

#### Load reference ECP image from Si[001] wafer

In [3]:
# Path to Si ECP reference pattern
si_ref_ecp_path = Path("../data/fcc_fe/")
si_ref_ecp_file = "01_Si_ref.tif"
si_ref_ecp_fname = os.path.join(si_ref_ecp_path, si_ref_ecp_file)

#### Load files that are relevant to the sample of interest (austenitic stainless steel polycrystal sample)
- EBSD master pattern of austenitic stainless steel at corresponding electron beam energy (20kV).
- EBSD map file (.ctf is used) from the steel sample.
- An overview SEM image from the EBSD mapping area.

In [4]:
# Path to fcc_Fe master pattern generated by EMsoft
fe_MP_path = Path("../data/ebsd_master_pattern/")   
fe_MP_file = "Fe-master-20kV.h5"
fe_MP_fname = os.path.join(fe_MP_path, fe_MP_file)
fe_MP = kp.load(fe_MP_fname, projection="lambert", hemisphere="both", energy=20)

# Path to fcc_Fe .ctf file containing the EBSD euler angles
fe_xmap_path = Path("../data/ebsd_map/")  
fe_xmap_file = "20kv_26nA_15mm WD_4x4 bin_fcc_Fe Map.ctf"
fe_xmap_fname = os.path.join(fe_xmap_path, fe_xmap_file)
fe_xmap = io.load_xmap(fe_xmap_fname)

# Reference overview image 
sem_path = Path("../data/fcc_fe/")  
sem_file = "01_steel overview.tif"
sem_fname = os.path.join(sem_path, sem_file)

#### Apply image correlation between EBSD IPF map and SEM overview image
This is an identical but simplified process that has been demonstrated in tutorial notebook [03_polycrystal_SEM_IPF_correlation](./03_polycrystal_SEM_IPF_correlation.ipynb).

In [5]:
correlation = optimization.ipf_image_correlation(sem_fname, fe_xmap_fname)
ipf_map = correlation.get_ipf_map(phase_name="Iron fcc", plot=False)
correlation.load_alignment_points(load_path="alignment_points.pkl")
ipf_warp_blended, _ = correlation.process(plot=False)

Phase     Orientations         Name  Space group  Point group  Proper point group     Color
   -1      8358 (2.5%)  not_indexed         None         None                None         w
    1   322978 (97.5%)     Iron fcc        Fm-3m         m-3m                 432  tab:blue
Properties: bands, error, MAD, BC, BS
Scan unit: um
24 alignment points have been loaded.


#### Interactive window to pick the region/grain of interest
Executing this cell to pop out an interactive QT window with the plot of blended EBSD IPF map with overview SEM image. Double-click at the grain of interest within the EBSD map area to position a virtual electron probe for simulation. The starting position of the virtual probe can be set using `initial_coord`. In this example, the initial position is set to the center of the blened image. Once a virtual electron probe position is selected, the pixel position in the EBSD IPF map, Euler angles, and absolute stage position of the clicked point are updated in the image tilte. 

__Note__: the displayed Euler angles in the title are the raw data stored in the corresponding EBSD file. When using them from computation, these angles need to be converted into TSL convention.

In [6]:
%matplotlib qt
coord_results = correlation.interactive_blended_xmap(initial_coord=[ipf_warp_blended.shape[1]//2, ipf_warp_blended.shape[0]//2])

Euler angles at the image point: [99.38360000000002, 5.765999999999999, 81.1616]
Phase name at the image point: <name: Iron fcc. space group: Fm-3m. point group: m-3m. proper point group: 432. color: tab:blue>


#### Interactive window to compute required stage tilt and rotation to two beam conditions
Run this cell will get the clicked region/grain of interest from the last cell, and utilize the Euler angles for the RKP simulations. In simulation, two RKP will be plotted. The RKP with smaller camera length (wider angular range) displays the overview of nearby Kikuchi bands with labels. A second RKP with slightly larger camera length (smaller angular range) displays the details close to the pattern center. The detailed RKP covers an angular range of approximately +-15 deg to guide the SEM stage tilt and rotation. Double-click on any point within the detailed RKP will trigger a calculation, which will suggest the recommended stage tilt and rotation to align the crystal direction to electron beam direction.

In [13]:
tiltX_corr_angle_op = optimized_correction_coefficients["tiltX_corr_angle"] # positive direction => pattern moves DOWN
tiltY_corr_angle_op = optimized_correction_coefficients["tiltY_corr_angle"] # positive direction => pattern moves LEFT
tiltZ_corr_angle_op = optimized_correction_coefficients["tiltZ_corr_angle"] # positive direction => pattern rotates COUNTER-CLOCKWISE
PCz_op = optimized_correction_coefficients["PCz"] # Larger value => smaller angular range

correlation.interactive_rkp(ref_ECP_path= si_ref_ecp_fname,
                            RKP_masterpattern=fe_MP,
                            corr_angles=[tiltX_corr_angle_op, tiltY_corr_angle_op, tiltZ_corr_angle_op],
                            convergence_angle=1,
                            semi_angle=False,
                            stage_mode="rot-tilt",
                            )

Finding bands that are in some pattern:
[########################################] | 100% Completed |  0.1s
Finding zone axes that are in some pattern:
[########################################] | 100% Completed |  0.1s
Calculating detector coordinates for bands and zone axes:
[########################################] | 100% Completed |  0.1s
